## k-Fold Cross Validation



Vai aplicar os testes ao método do Kernel SVM

In [1]:
# k-Fold Cross Validation

# Importing the dataset
dataset = read.csv('Social_Network_Ads.csv')
dataset = dataset[3:5]


In [2]:
# Encoding the target feature as factor
dataset$Purchased = factor(dataset$Purchased, levels = c(0, 1))


In [3]:
# Splitting the dataset into the Training set and Test set
# install.packages('caTools')
library(caTools)
set.seed(123)
split = sample.split(dataset$Purchased, SplitRatio = 0.75)
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)


In [4]:
# Feature Scaling
training_set[-3] = scale(training_set[-3])
test_set[-3] = scale(test_set[-3])


In [5]:
# Fitting Kernel SVM to the Training set
# install.packages('e1071')
library(e1071)
classifier = svm(formula = Purchased ~ .,
                 data = training_set,
                 type = 'C-classification',
                 kernel = 'radial')


In [6]:
# Predicting the Test set results
y_pred = predict(classifier, newdata = test_set[-3])



In [7]:
# Making the Confusion Matrix
cm = table(test_set[, 3], y_pred)


In [8]:
cm


   y_pred
     0  1
  0 58  6
  1  4 32

In [9]:
# Applying k-Fold Cross Validation
install.packages('caret')



The downloaded binary packages are in
	/var/folders/qq/gkyyxk3d0q769zkwc0hf1mfw0000gn/T//RtmpJl2TGu/downloaded_packages


In [11]:
library(caret)



Warning message:
“package ‘caret’ was built under R version 3.4.4”Loading required package: lattice
Loading required package: ggplot2


In [12]:
folds = createFolds(training_set$Purchased, k = 10)


In [13]:
folds

$Fold01
 [1]  21  50  59  62  85  88  93 107 108 116 130 142 146 156 160 168 195 222 227
[20] 236 238 241 243 244 251 256 265 283 292 294

$Fold02
 [1]   1  15  20  23  34  53  57  61  81  87  94 113 123 133 144 147 161 178 181
[20] 186 247 252 274 276 279 285 289 295 296 297 300

$Fold03
 [1]  18  28  32  41  58  71  73  80  84  89  90  96 105 106 127 129 137 140 153
[20] 158 159 167 179 206 217 220 246 269 278

$Fold04
 [1]   7  10  16  29  39  64  66  67  83  86  98  99 114 124 126 131 136 139 155
[20] 212 226 239 248 249 255 257 260 281 284 288

$Fold05
 [1]   4   5  13  22  26  31  37  40  49  51  56  70  91  92  95  97 102 117 164
[20] 166 180 240 253 267 271 273 282 286 298

$Fold06
 [1]   2   3   6  14  30  35  48  65  72  75  76 112 119 122 128 154 165 171 172
[20] 189 191 194 196 198 204 205 214 228 234 254

$Fold07
 [1]  27  36  46  52  54  55  60  79  82 100 110 120 132 135 157 169 177 185 209
[20] 211 215 218 231 258 259 261 262 263 272 293

$Fold08
 [1]   8  17  24  33  44  45  74 103 104 109 111 115 118 134 138 141 143 145 152
[20] 163 173 174 197 208 216 219 229 232 235 268 270

$Fold09
 [1]   9  12  38  42  43  47  63  68 101 121 148 149 151 182 183 192 201 203 207
[20] 221 223 230 245 266 275 277 280 287 290 299

$Fold10
 [1]  11  19  25  69  77  78 125 150 162 170 175 176 184 187 188 190 193 199 200
[20] 202 210 213 224 225 233 237 242 250 264 291

In [18]:
head(training_set,10)

,Age,EstimatedSalary,Purchased
1,-1.7655475,-1.4733414,0
3,-1.0962966,-0.7883761,0
6,-1.0006894,-0.3602727,0
7,-1.0006894,0.3817730,0
8,-0.5226531,2.2654277,1
10,-0.2358313,-0.1604912,0
11,-1.0962966,0.2676121,0
13,-1.6699402,0.4388535,0
14,-0.5226531,-1.5018816,0
15,-1.8611548,0.3246926,0


In [15]:
cv = lapply(folds, function(x) {
  training_fold = training_set[-x, ]
  test_fold = training_set[x, ]
  classifier = svm(formula = Purchased ~ .,
                   data = training_fold,
                   type = 'C-classification',
                   kernel = 'radial')
  y_pred = predict(classifier, newdata = test_fold[-3])
  cm = table(test_fold[, 3], y_pred)
  accuracy = (cm[1,1] + cm[2,2]) / (cm[1,1] + cm[2,2] + cm[1,2] + cm[2,1])
  return(accuracy)
})

In [19]:
cv

$Fold01
[1] 0.9333333

$Fold02
[1] 0.8709677

$Fold03
[1] 1

$Fold04
[1] 0.8666667

$Fold05
[1] 0.9655172

$Fold06
[1] 0.9

$Fold07
[1] 0.9

$Fold08
[1] 0.9354839

$Fold09
[1] 0.9

$Fold10
[1] 0.8333333

In [16]:
accuracy = mean(as.numeric(cv))
accuracy

[1] 0.9105302

A função lapply faz um referência de folds para o x da "function"

training_fold = training_set[-x, ] usa todos os dados menos o fold corrente. Usa esse cara que fica de fora é usado para o teste.

Definido os dados...ainda dentro da função ele estabelece o "classifier"svm com o training_fold. Determina o vetor de dados previsto.....faz a matriz de confusão....e determina a acurácia para cada um dos "folds"


